In [ ]:
import os
from ultralytics import YOLO
import yaml
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
from source.visual_genome_meta_data import read_json_to_dict
from source.visual_genome_meta_data import get_image_meta_data
from source.visual_genome_data import count_occurrences
from source.visual_genome_to_yolo import create_class_mapping_from_list
from source.visual_genome_to_yolo import save_class_map_to_yaml
from source.visual_genome_to_yolo import convert_single_image_to_yolo
from source.visual_genome_to_yolo import read_yaml_to_class_map
from source.visual_genome_to_yolo import read_yolo_metadata
from source.visual_genome_to_yolo import visual_genome_to_yolo_data_n
from source.visual_genome_meta_data import plot_image_with_multiple_bboxes
from source.visual_genome_data import get_image_ids

In [ ]:
import os
import yaml
import torch
from ultralytics import YOLO

In [ ]:
from PIL import Image

In [ ]:
def train_yolo_on_single_class(
    data_dir,          # Directory with your dataset
    class_yaml_path,   # Path to your class_map.yaml file
    target_class,      # The single class to train on (e.g., 'mountain')
    output_dir,        # Directory to save results
    epochs=50,         # Number of training epochs
    img_size=640,      # Input image size
    batch_size=16,     # Batch size
    device=None        # Device for training - will auto-select MPS if available
):
    """
    Train YOLO v11 on a single class from your dataset using M1 Mac's MPS acceleration.
    """
    # import os
    # import yaml
    # import torch
    # from ultralytics import YOLO  # Assuming this is the import for YOLOv11
    
    # Auto-select the best available device with priority for MPS on Mac
    if device is None:
        if torch.backends.mps.is_available():
            device = 'mps'
            print(f"Using MPS acceleration on Apple Silicon")
        elif torch.cuda.is_available():
            device = 'cuda'
            print(f"Using CUDA acceleration")
        else:
            device = 'cpu'
            print(f"Using CPU for training (this will be slow)")
    
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # 1. Load the full class map
    with open(class_yaml_path, 'r') as f:
        class_data = yaml.safe_load(f)
    
    # Find the index of the target class
    all_classes = class_data['names']
    if target_class not in all_classes:
        raise ValueError(f"Class '{target_class}' not found in class map. Available classes: {all_classes}")
    
    target_class_id = all_classes.index(target_class)
    print(f"Training on class: {target_class} (class_id: {target_class_id})")
    
    # 2. Create a new YAML file for single-class training
    single_class_yaml = os.path.join(output_dir, f"single_class_{target_class}.yaml")
    
    # Update dataset configuration
    train_path = os.path.join(data_dir, 'train/images')
    val_path = os.path.join(data_dir, 'val/images')
    
    single_class_config = {
        'path': os.path.abspath(data_dir),
        'train': 'train/images',
        'val': 'val/images',
        'nc': 1,  # Just one class
        'names': [target_class]  # Only the target class
    }
    
    with open(single_class_yaml, 'w') as f:
        yaml.dump(single_class_config, f)
    
    print(f"Created single-class configuration at: {single_class_yaml}")
    
    # 3. Initialize YOLO model
    model = YOLO("yolo11n.pt")  # Using YOLOv11 nano (smallest model)
    
    # 4. Train the model with MPS-specific configurations
    results = model.train(
        data=single_class_yaml,
        epochs=epochs,
        imgsz=img_size,
        batch=batch_size,
        project=output_dir,
        name=f"yolov11_{target_class}",
        exist_ok=True,
        patience=0,  # Early stopping
        verbose=True,
        device=device,  # Specify device for training
        amp=True       # Enable mixed precision training (improves performance on M1/M2)
    )
    
    print(f"Training completed. Model saved to {output_dir}/yolov11_{target_class}")
    return results

In [ ]:
root_path = Path('/Users/stephanehess/Documents/CAS_AML/dias_digit_project')
data_dir = root_path / 'yolo_object_train'
class_yaml_path = data_dir / 'class_map.yaml'
target_class = 'mountain'
#target_class = 'church'
output_dir = data_dir / 'output'



In [ ]:

# Train the model
training_results = train_yolo_on_single_class(
    data_dir=data_dir,
    class_yaml_path=class_yaml_path,
    target_class=target_class,
    output_dir=output_dir,
    epochs=30,
    img_size=640,
    batch_size=26  # Adjust based on your GPU memory
)

In [ ]:
#model_path = data_dir / 'output/yolov11_mountain/weights/best.pt'
#model_path = data_dir / 'output/yolov11_church/weights/best.pt'
model_path = data_dir / 'output/yolov11_church/weights/last.pt'


In [ ]:
model_path

In [ ]:
test_images_dir = data_dir / 'test'
test_images_dir

In [ ]:
os.listdir(test_images_dir)

In [ ]:


def test_yolo_model(model_path, test_images_dir, conf_threshold=0.3):
    """
    Test a trained YOLO model on a directory of test images.
    
    Args:
        model_path: Path to the trained model weights (.pt file)
        test_images_dir: Directory containing test images
        conf_threshold: Confidence threshold for detections
    """
    # Load the trained model
    model = YOLO(model_path)
    
    # Get list of test images
    test_images = [f for f in os.listdir(test_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tif'))]
    
    # Summary statistics
    total_images = len(test_images)
    images_with_detections = 0
    total_detections = 0
    
    # Process each image
    for i, img_file in enumerate(test_images):
        img_path = os.path.join(test_images_dir, img_file)
        
        # Run inference
        results = model(img_path, conf=conf_threshold)
        
        # Get detection results
        detections = results[0].boxes
        
        # Count detections
        if len(detections) > 0:
            images_with_detections += 1
            total_detections += len(detections)
        
        # Display the image with detections (every 5th image)
        if i % 5 == 0:
            # Get the image
            img = Image.open(img_path)
            
            # Create figure and axis
            fig, ax = plt.subplots(figsize=(10, 10))
            ax.imshow(np.array(img))
            
            # Add detections to the plot
            for det in detections:
                # Get bounding box coordinates (x1, y1, x2, y2)
                box = det.xyxy[0].cpu().numpy()
                
                # Create rectangle patch
                rect = patches.Rectangle(
                    (box[0], box[1]), 
                    box[2] - box[0], 
                    box[3] - box[1], 
                    linewidth=2, 
                    edgecolor='red', 
                    facecolor='none'
                )
                ax.add_patch(rect)
                
                # Add class name and confidence
                cls_id = int(det.cls[0])
                conf = float(det.conf[0])
                class_name = model.names[cls_id]
                ax.text(
                    box[0], box[1] - 5, 
                    f"{class_name}: {conf:.2f}", 
                    color='red', 
                    fontsize=12, 
                    bbox=dict(facecolor='white', alpha=0.7)
                )
            
            # Set title and display
            ax.set_title(f"Image {i+1}/{total_images}: {len(detections)} detections")
            plt.axis('off')
            plt.tight_layout()
            plt.show()
            plt.close()
        
        # Print progress
        if (i+1) % 10 == 0:
            print(f"Processed {i+1}/{total_images} images")
    
    # Print summary statistics
    print(f"\nTest Results Summary:")
    print(f"Total images: {total_images}")
    print(f"Images with detections: {images_with_detections} ({images_with_detections/total_images*100:.1f}%)")
    print(f"Total detections: {total_detections}")
    print(f"Average detections per image: {total_detections/total_images:.2f}")
    
    return results


In [ ]:

# Example usage
confidence = 0.1                         # Confidence threshold

# Run the test
test_results = test_yolo_model(model_path, test_images_dir, confidence)

In [ ]:
test_results

In [ ]:
model_path

In [ ]:
def plot_detections_on_test_images(
    trained_model_path,  # Path to the trained model weights (best.pt)
    test_images_dir,     # Directory containing test images
    output_dir,          # Directory to save visualized results
    conf_threshold=0.25, # Confidence threshold for detections
    device=None,         # Device to run inference on (will use MPS if available)
    img_size=640         # Size for inference
):
    """
    Use trained YOLOv11 model to detect objects in test images and visualize results.
    """
    import os
    import glob
    import torch
    from ultralytics import YOLO
    from pathlib import Path
    
    # Auto-select device
    if device is None:
        if torch.backends.mps.is_available():
            device = 'mps'
            print(f"Using MPS acceleration on Apple Silicon")
        elif torch.cuda.is_available():
            device = 'cuda'
            print(f"Using CUDA acceleration")
        else:
            device = 'cpu'
            print(f"Using CPU for inference")
    
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Load the trained model
    model = YOLO(trained_model_path)
    
    # Get list of test images
    test_images = []
    for ext in ['jpg', 'jpeg', 'png', 'bmp', 'tif']:
        test_images.extend(glob.glob(os.path.join(test_images_dir, f'*.{ext}')))
    
    print(f"Found {len(test_images)} test images")
    
    # Process each test image
    for img_path in test_images:
        # Get filename without extension
        img_name = Path(img_path).stem
        
        # Run inference on image
        results = model.predict(
            source=img_path,
            conf=conf_threshold,
            device=device,
            imgsz=img_size,
            save=True,       # Save images with detections
            save_txt=True,   # Save results as .txt files (optional)
            project=output_dir,
            name='detections',
            exist_ok=True
        )
        
        print(f"Processed {img_name}: {len(results[0].boxes)} detections")
    
    print(f"Detection results saved to {os.path.join(output_dir, 'detections')}")
    print(f"- Images with bounding boxes saved in {os.path.join(output_dir, 'detections')}")
    print(f"- Detection data saved in {os.path.join(output_dir, 'detections/labels')}")

    return os.path.join(output_dir, 'detections')

In [ ]:
output_dir = data_dir / 'output_test'
confidence = 0.25

In [ ]:
test_images_dir

In [ ]:
output_dir

In [ ]:
model_path

In [ ]:
results_dir = plot_detections_on_test_images(model_path, test_images_dir, output_dir, confidence)

In [ ]:
results_dir